# Process Substitution

---

In the previous section we've seen how to chain output of one command to the next one. But what if you want to chain the output of two or more commands to another one? What if you have a command that takes a file as argument but you would like to process whatever is sent to that file?

Process substitution allows a process's input or output to be referred to using a filename. It has two forms:

- Output: `<(cmd)`
- Input: `>(cmd)`

## Output Process Substitution: `<(cmd)`

Imagine you've two files for which you want to compare the content. Using `diff file1 file2` could generate false positives in the case lines are not ordered. So if you want to compare those files you could create two new files, ordered, and compare those. It would look like:

```bash
sort file1 > sorted_file1
sort file2 > sorted_file2
diff sorted_file1 sorted_file2
```

With process substitution you can do it in one line:

```bash
diff <(sort file1) <(sort file2)
```

### Example 1: Comparing sorted files

In [1]:
# Create two test files with unsorted content
cat > /tmp/file1.txt << EOF
banana
apple
cherry
EOF

cat > /tmp/file2.txt << EOF
cherry
banana
apple
EOF

echo "Files are identical when sorted:"
diff <(sort /tmp/file1.txt) <(sort /tmp/file2.txt) && echo "No differences!" || echo "Files differ"

Files are identical when sorted:
No differences!
No differences!


### Example 2: Comparing command outputs

In [2]:
# Compare list of files in two directories
mkdir -p /tmp/dir1 /tmp/dir2
touch /tmp/dir1/{a,b,c}.txt
touch /tmp/dir2/{b,c,d}.txt

echo "Files only in dir1 or dir2:"
diff <(ls /tmp/dir1) <(ls /tmp/dir2)

Files only in dir1 or dir2:
1d0
< a.txt
3a3
> d.txt
1d0
< a.txt
3a3
> d.txt


: 1

### Example 3: Join command with process substitution

In [3]:
# Join two sorted command outputs
cat > /tmp/names.txt << EOF
1 Alice
2 Bob
3 Charlie
EOF

cat > /tmp/ages.txt << EOF
1 25
2 30
3 35
EOF

echo "Joined data:"
join <(sort /tmp/names.txt) <(sort /tmp/ages.txt)

Joined data:
1 Alice 25
2 Bob 30
3 Charlie 35
1 Alice 25
2 Bob 30
3 Charlie 35


## Input Process Substitution: `>(cmd)`

Imagine you want to store logs of an application into a file and at the same time print it on the console. A very handy command for that is `tee`.

```bash
echo "Hello, world!" | tee /tmp/hello.txt
```

Now let say you want to have only lower case characters in the file but keep the regular case on the output. You could use process substitution that way:

```bash
echo "Hello, world!" | tee >(tr '[:upper:]' '[:lower:]' > /tmp/hello.txt)
```

### Example 4: Transform output before saving

In [4]:
echo "Hello, World!" | tee >(tr '[:upper:]' '[:lower:]' > /tmp/hello.txt)
echo "Original output shown above"
echo "File contains:"
cat /tmp/hello.txt

Hello, World!
Original output shown above
Original output shown above
File contains:
File contains:
hello, world!
hello, world!


### Example 5: Send output to multiple processes

In [5]:
# Send output to multiple files with different transformations
echo "Testing Multiple Outputs" | tee \
    >(tr '[:upper:]' '[:lower:]' > /tmp/lower.txt) \
    >(tr '[:lower:]' '[:upper:]' > /tmp/upper.txt) \
    > /tmp/original.txt

sleep 0.1  # Give time for background processes

echo "Original:"
cat /tmp/original.txt
echo "Lowercase:"
cat /tmp/lower.txt
echo "Uppercase:"
cat /tmp/upper.txt

Original:
Testing Multiple Outputs
Testing Multiple Outputs
Lowercase:
Lowercase:
testing multiple outputs
testing multiple outputs
Uppercase:
Uppercase:
TESTING MULTIPLE OUTPUTS
TESTING MULTIPLE OUTPUTS


## Practical Examples

### Example 6: Comparing filtered logs

In [6]:
# Create sample log files
cat > /tmp/log1.txt << EOF
2023-10-26 ERROR: Connection failed
2023-10-26 INFO: Starting service
2023-10-26 ERROR: Database timeout
2023-10-26 INFO: Service running
EOF

cat > /tmp/log2.txt << EOF
2023-10-26 ERROR: Database timeout
2023-10-26 INFO: Service running
2023-10-26 ERROR: Connection failed
2023-10-26 WARN: High memory usage
EOF

echo "Comparing ERROR messages:"
diff <(grep ERROR /tmp/log1.txt | cut -d' ' -f3-) <(grep ERROR /tmp/log2.txt | cut -d' ' -f3-)

Comparing ERROR messages:
1d0
< Connection failed
2a2
> Connection failed
1d0
< Connection failed
2a2
> Connection failed


: 1

### Example 7: Creating archive with processed content

In [7]:
# Compress output of a command directly
echo "Creating processed archive"
find /tmp -name "*.txt" -type f 2>/dev/null | head -5 | tar czf /tmp/files.tar.gz -T -
echo "Archive created:"
ls -lh /tmp/files.tar.gz

Creating processed archive
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
Archive created:
Archive created:
-rw-rw-r-- 1 bigalex95 bigalex95 214 Eki 26 01:06 /tmp/files.tar.gz
-rw-rw-r-- 1 bigalex95 bigalex95 214 Eki 26 01:06 /tmp/files.tar.gz


### Example 8: Multiple input sources

In [8]:
# Combine output from multiple sources
cat > /tmp/data1.txt << EOF
apple
banana
EOF

cat > /tmp/data2.txt << EOF
cherry
date
EOF

echo "Combining and sorting multiple files:"
sort <(cat /tmp/data1.txt) <(cat /tmp/data2.txt) <(echo "elderberry")

Combining and sorting multiple files:
apple
banana
cherry
date
elderberry
apple
banana
cherry
date
elderberry


## Advanced Use Cases

In [9]:
# Parallel processing with process substitution
paste <(seq 1 5) <(seq 10 14) <(seq 20 24)

1	10	20
2	11	21
3	12	22
4	13	23
5	14	24
2	11	21
3	12	22
4	13	23
5	14	24


In [10]:
# Compare output of two different commands
diff <(ps aux | grep bash | awk '{print $2}') <(pgrep bash) || echo "PIDs don't match perfectly"

1,2d0
< 38969
< 205949
4d1
< 207034
6d2
< 212147
8d3
< 212559
10d4
< 213431
12d5
< 213797
14d6
< 214472
16d7
< 217375
19c10
< 217718
---
> 217715
PIDs don't match perfectly
< 38969
< 205949
4d1
< 207034
6d2
< 212147
8d3
< 212559
10d4
< 213431
12d5
< 213797
14d6
< 214472
16d7
< 217375
19c10
< 217718
---
> 217715
PIDs don't match perfectly


## Note

There is no exercise for this section. Process substitution is an advanced feature that's useful in specific scenarios. Practice using it when you need to:

- Compare outputs of commands
- Feed command output to programs expecting files
- Process data in parallel
- Avoid creating temporary files